*This version has no duplicated words in test dataset*

In [ ]:
! pip install datasets transformers --upgrade

# Extract synonyms that the tokenizer supports
Load the *Worndnet* data

In [2]:
import pandas as pd
from nltk.corpus import wordnet as wn
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

Load the Tokenizer

In [3]:
from transformers import AutoTokenizer

model_checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
tokenizer

PreTrainedTokenizerFast(name_or_path='bert-base-uncased', vocab_size=30522, model_max_len=512, is_fast=True, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

Get through the words in the tokenizer

In [4]:
len(tokenizer.get_vocab())

30522

In [5]:
synonyms = []

for synset in list(wn.all_synsets()):
  temp = synset.lemma_names()[:]
  if len(temp)<=1:
    continue
  temp2 = []
  for i in temp:
    if len(tokenizer(i)['input_ids'])==3:
      temp2.append(i)
  if len(temp2)<=1:
    continue  
  synonyms.append(temp2)

In [6]:
len(synonyms)

7747

Extract words

In [7]:
from itertools import combinations

df = []
for line in synonyms:
  comb = combinations(line, 2)
  for i in list(comb):
    temp1 = line[:]
    temp1.remove(i[0])
    temp2 = line[:]
    temp2.remove(i[1])
    df.append({
      'word1': i[0],
      'word2': i[1],
      'synonyms1': temp1,
      'synonyms2': temp2,
    })

In [8]:
df[:10]

[{'synonyms1': ['shortened'],
  'synonyms2': ['cut'],
  'word1': 'cut',
  'word2': 'shortened'},
 {'synonyms1': ['comparative'],
  'synonyms2': ['relative'],
  'word1': 'relative',
  'word2': 'comparative'},
 {'synonyms1': ['tangible'],
  'synonyms2': ['real'],
  'word1': 'real',
  'word2': 'tangible'},
 {'synonyms1': ['rich'],
  'synonyms2': ['ample'],
  'word1': 'ample',
  'word2': 'rich'},
 {'synonyms1': ['faithful'],
  'synonyms2': ['close'],
  'word1': 'close',
  'word2': 'faithful'},
 {'synonyms1': ['outside'],
  'synonyms2': ['away'],
  'word1': 'away',
  'word2': 'outside'},
 {'synonyms1': ['incorrect', 'wrong'],
  'synonyms2': ['faulty', 'wrong'],
  'word1': 'faulty',
  'word2': 'incorrect'},
 {'synonyms1': ['incorrect', 'wrong'],
  'synonyms2': ['faulty', 'incorrect'],
  'word1': 'faulty',
  'word2': 'wrong'},
 {'synonyms1': ['faulty', 'wrong'],
  'synonyms2': ['faulty', 'incorrect'],
  'word1': 'incorrect',
  'word2': 'wrong'},
 {'synonyms1': ['recognized', 'recognised'],
  

In [9]:
len(df)

15586

# Generate sentences
## Version 1: unidirection

In [10]:
# datasets = []

# for line in df:
#   sen = f'{line[0]} is close in meaning to {line[1]}.'
#   word = line[1]
#   datasets.append({'sen':sen, 'word':word})

# datasets[:10]

In [11]:
# len(datasets)

## Version 2: bi-direction

In [12]:
datasets = []

for line in df:
  sen = f'{line["word1"]} is close in meaning to {line["word2"]}.'
  datasets.append({'sen':sen, 'word1':line["word1"], 'word2':line["word2"], 'synonyms':line['synonyms1']})
  sen = f'{line["word2"]} is close in meaning to {line["word1"]}.'
  datasets.append({'sen':sen, 'word1':line["word2"], 'word2':line["word1"], 'synonyms':line['synonyms2']})

datasets[:20]

[{'sen': 'cut is close in meaning to shortened.',
  'synonyms': ['shortened'],
  'word1': 'cut',
  'word2': 'shortened'},
 {'sen': 'shortened is close in meaning to cut.',
  'synonyms': ['cut'],
  'word1': 'shortened',
  'word2': 'cut'},
 {'sen': 'relative is close in meaning to comparative.',
  'synonyms': ['comparative'],
  'word1': 'relative',
  'word2': 'comparative'},
 {'sen': 'comparative is close in meaning to relative.',
  'synonyms': ['relative'],
  'word1': 'comparative',
  'word2': 'relative'},
 {'sen': 'real is close in meaning to tangible.',
  'synonyms': ['tangible'],
  'word1': 'real',
  'word2': 'tangible'},
 {'sen': 'tangible is close in meaning to real.',
  'synonyms': ['real'],
  'word1': 'tangible',
  'word2': 'real'},
 {'sen': 'ample is close in meaning to rich.',
  'synonyms': ['rich'],
  'word1': 'ample',
  'word2': 'rich'},
 {'sen': 'rich is close in meaning to ample.',
  'synonyms': ['ample'],
  'word1': 'rich',
  'word2': 'ample'},
 {'sen': 'close is close in 

In [13]:
len(datasets)

31172

# Creat datasets

In [14]:
from numpy import random, logical_not

random.seed(123)

# Random pick some words as test data
datasets_df = pd.DataFrame(datasets)
temp = datasets_df['word1'].unique()
test_words = random.choice(temp, replace=False, size=round(0.3*temp.size))

# Extract those words
flag = datasets_df['word1'].isin(test_words)
test = datasets_df.loc[flag,:].reset_index(drop=True)
train = datasets_df.loc[logical_not(flag),:].reset_index(drop=True)

In [15]:
print(test_words.size)
print(train.shape[0])

2496
21952


In [16]:
test.head()

,sen,word1,word2,synonyms
0,real is close in meaning to tangible.,real,tangible,[tangible]
1,ample is close in meaning to rich.,ample,rich,[rich]
2,faithful is close in meaning to close.,faithful,close,[close]
3,away is close in meaning to outside.,away,outside,[outside]
4,outside is close in meaning to away.,outside,away,[away]


In [17]:
train.head()

,sen,word1,word2,synonyms
0,cut is close in meaning to shortened.,cut,shortened,[shortened]
1,shortened is close in meaning to cut.,shortened,cut,[cut]
2,relative is close in meaning to comparative.,relative,comparative,[comparative]
3,comparative is close in meaning to relative.,comparative,relative,[relative]
4,tangible is close in meaning to real.,tangible,real,[real]


Clear test dataset, word1 can only appear once

In [18]:
test = test.groupby('word1').agg({'sen':'first', 'synonyms': sum, 'word2':'first'}).reset_index()

def get_syn(row):
  synonyms = []
  for syn in wn.synsets(row):
    for l in syn.lemmas():
      if len(tokenizer(l.name())['input_ids'])==3:
        synonyms.append(l.name())
  synonyms = list(set(synonyms))
  synonyms.remove(row)
  return synonyms

test['synonyms'] = test['word1'].apply(lambda x: get_syn(x))

test.iloc[250:270,:]

,word1,sen,synonyms,word2
250,agricultural,agricultural is close in meaning to agrarian.,"[farming, agrarian]",agrarian
251,aided,aided is close in meaning to assisted.,"[aid, help, assisted, assist]",assisted
252,ain,ain is close in meaning to own.,[own],own
253,airfield,airfield is close in meaning to field.,[field],field
254,airlift,airlift is close in meaning to lift.,[lift],lift
255,airplane,airplane is close in meaning to plane.,[plane],plane
256,akin,akin is close in meaning to kin.,[kin],kin
257,align,align is close in meaning to adjust.,"[adjust, array, coordinate]",adjust
258,alive,alive is close in meaning to live.,"[awake, active, alert, live, animated]",live
259,alliance,alliance is close in meaning to confederation.,"[bond, alignment, coalition, confederation]",confederation


In [19]:
from datasets import DatasetDict, Dataset

datasets = DatasetDict({
    'train': Dataset.from_pandas(train),
    'test': Dataset.from_pandas(test)})

In [20]:
datasets

DatasetDict({
    train: Dataset({
        features: ['sen', 'word1', 'word2', 'synonyms'],
        num_rows: 21952
    })
    test: Dataset({
        features: ['word1', 'sen', 'synonyms', 'word2'],
        num_rows: 2496
    })
})

# Tokenize

In [21]:
def findindex(seq, subseq):
  # brute-force approach O(n*m)
  # Usage: findindex([4,3,'a',5,6], [5,6])
  i, n, m = -1, len(seq), len(subseq)
  try:
    while True:
      i = seq.index(subseq[0], i + 1, n - m + 1)
      if subseq == seq[i:i + m]:
        return i
  except ValueError:
    return -1

In [22]:
def tokenize_function(examples):
  if length==0:
    sen_no_mask = tokenizer(examples["sen"], truncation=True, padding=True)
  else:
    sen_no_mask = tokenizer(examples["sen"], truncation=True, padding='max_length', max_length=length)
  masked_word = tokenizer(examples["word2"], truncation=True, padding=True)['input_ids']
  inputs = []
  labels = []
  
  for i in range(len(masked_word)):
    # Find the word encoding part
    start = masked_word[i].index(101)
    end = masked_word[i].index(102)
    # Extract this part
    temp = [masked_word[i][j] for j in range(start+1,end)]
    # Find the same part in input and mask them
    ipt = sen_no_mask['input_ids'][i][:]
    idx = findindex(ipt, temp)
    for j in range(len(temp)):
      ipt[idx+j] = tokenizer.mask_token_id
    inputs.append(ipt)
    # Find the other part and replace those unmasked indices with -100
    label = sen_no_mask['input_ids'][i][:]
    for j in range(len(ipt)):
      if ipt[j]!=tokenizer.mask_token_id:
        label[j]=-100
    labels.append(label)
  
  # encode synonyms
  synonyms = []
  for i in examples["synonyms"]:
    temp = tokenizer(i)['input_ids']
    synonym = []
    for j in temp:
      synonym.append(j[1])
    synonyms.append(synonym)

  sen_no_mask['input_ids']=inputs
  sen_no_mask['label']=labels
  sen_no_mask['synonyms']=synonyms
  return sen_no_mask

length = 0
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["sen", "word2", "word1"])

In [23]:
for i in datasets['test']['synonyms'][0]:
  print(tokenizer(i)['input_ids'])

[101, 1047, 102]
[101, 2882, 102]
[101, 1043, 102]
[101, 4220, 102]
[101, 1049, 102]
[101, 1049, 102]
[101, 1047, 102]
[101, 15223, 102]
[101, 4595, 102]


In [24]:
for i in range(5):
  print(tokenizer.decode(tokenized_datasets['train']['input_ids'][i]))
  print(datasets['train']['sen'][i])
  print(tokenized_datasets['train']['synonyms'][i])
  print(datasets['train']['synonyms'][i])

[CLS] cut is close in meaning to [MASK]. [SEP]
cut is close in meaning to shortened.
[12641]
['shortened']
[CLS] shortened is close in meaning to [MASK]. [SEP]
shortened is close in meaning to cut.
[3013]
['cut']
[CLS] relative is close in meaning to [MASK]. [SEP]
relative is close in meaning to comparative.
[12596]
['comparative']
[CLS] comparative is close in meaning to [MASK]. [SEP]
comparative is close in meaning to relative.
[5816]
['relative']
[CLS] tangible is close in meaning to [MASK]. [SEP]
tangible is close in meaning to real.
[2613]
['real']


In [25]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'input_ids', 'label', 'synonyms', 'token_type_ids'],
        num_rows: 21952
    })
    test: Dataset({
        features: ['attention_mask', 'input_ids', 'label', 'synonyms', 'token_type_ids'],
        num_rows: 2496
    })
})

# Train

In [26]:
from transformers import DataCollatorWithPadding
collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [27]:
from transformers import AutoConfig, AutoModelForMaskedLM

model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [28]:
model_raw = AutoModelForMaskedLM.from_pretrained(model_checkpoint)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [29]:
from transformers import pipeline

camembert_fill_mask  = pipeline("fill-mask", model=model_raw, tokenizer=tokenizer)
print(tokenizer.decode(tokenized_datasets['test']['input_ids'][287]))

results = camembert_fill_mask(tokenizer.decode(tokenized_datasets['test']['input_ids'][287]))
results

[CLS] approved is close in meaning to [MASK]. [SEP]


[{'score': 0.04130351543426514,
  'sequence': 'approved is close in meaning to english.',
  'token': 2394,
  'token_str': 'english'},
 {'score': 0.011473696678876877,
  'sequence': 'approved is close in meaning to latin.',
  'token': 3763,
  'token_str': 'latin'},
 {'score': 0.007926260121166706,
  'sequence': 'approved is close in meaning to earth.',
  'token': 3011,
  'token_str': 'earth'},
 {'score': 0.007475481368601322,
  'sequence': 'approved is close in meaning to it.',
  'token': 2009,
  'token_str': 'it'},
 {'score': 0.004972452763468027,
  'sequence': 'approved is close in meaning to greek.',
  'token': 3306,
  'token_str': 'greek'}]

### Check the performance of original PM
Accurcy is defined as if the true answer is in its top N predictions.

In [31]:
import torch

N = 5

In [32]:
import operator
from functools import reduce

def common_member(a, b):
  a_set = set(a)
  b_set = set(b)
  if (a_set & b_set):
    return True 
  else:
    return False

def check_accuracy(df, lb, outputs, n=5):
  mask_token_index = torch.where(df['input_ids'] == tokenizer.mask_token_id)[1]
  mask_token_logits = outputs.logits[range(outputs.logits.size()[0]), mask_token_index, :]
  top_n_tokens = torch.topk(mask_token_logits, n, dim=1).indices.tolist()
  return [1 if common_member(i, top_n_tokens[j]) else 0 for j,i in enumerate(lb)]

def get_accuracy(df, shards):
  accuracies = []
  for i in range(shards):
    pm_inputs = df.shard(num_shards=shards, index=i)
    test_inputs = dict((k, torch.LongTensor(pm_inputs[k])) for k in ['attention_mask', 'input_ids', 'token_type_ids'])
    test_labels = torch.LongTensor(pm_inputs['label'])
    outputs = model(**test_inputs, labels=test_labels)
    accuracy = check_accuracy(test_inputs, pm_inputs['synonyms'], outputs, N)
    accuracies.append(accuracy)
  return reduce(operator.concat, accuracies)

In [33]:
accuracy = get_accuracy(tokenized_datasets['test'], 20)

The accuracy is: `sum(accuracy)/len(accuracy)`

In [34]:
sum(accuracy)/len(accuracy)

0.15224358974358973

In [35]:
from transformers import Trainer, TrainingArguments
from math import ceil

batch_size=16
logging_steps=ceil(tokenized_datasets['train'].num_rows/batch_size)
num_train_epochs=5

training_args = TrainingArguments(
    "test-clm",
    evaluation_strategy = "epoch",
    learning_rate=2e-6,
    weight_decay=0.01,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_train_epochs,
    logging_steps=logging_steps,
    save_steps=3000,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=collator
)

trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: synonyms.
***** Running training *****
  Num examples = 21952
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 6860


Epoch,Training Loss,Validation Loss
1,6.802600,5.952183
2,6.275200,5.829013
3,6.048400,5.781848
4,5.920600,5.764913
5,5.852800,5.764584


The following columns in the evaluation set  don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: synonyms.
***** Running Evaluation *****
  Num examples = 2496
  Batch size = 16
The following columns in the evaluation set  don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: synonyms.
***** Running Evaluation *****
  Num examples = 2496
  Batch size = 16
Saving model checkpoint to test-clm/checkpoint-3000
Configuration saved in test-clm/checkpoint-3000/config.json
Model weights saved in test-clm/checkpoint-3000/pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: synonyms.
***** Running Evaluation *****
  Num examples = 2496
  Batch size = 16
The following columns in the evaluation set  don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: synonyms.
***** Running Evaluation *****
  Num 

TrainOutput(global_step=6860, training_loss=6.179916550883746, metrics={'train_runtime': 452.7878, 'train_samples_per_second': 242.409, 'train_steps_per_second': 15.151, 'total_flos': 564245317440000.0, 'train_loss': 6.179916550883746, 'epoch': 5.0})

### Take a look at the test

In [36]:
j=287
testdata = tokenized_datasets['test'].select([j]).remove_columns("label")

In [37]:
for i in [j]:
  print(tokenizer.decode(tokenized_datasets['test']['input_ids'][i]))
  print(datasets['test']['synonyms'][i])

[CLS] approved is close in meaning to [MASK]. [SEP]
['approve', 'okay', 'sanctioned']


In [38]:
temp = trainer.predict(testdata)

The following columns in the test set  don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: synonyms.
***** Running Prediction *****
  Num examples = 1
  Batch size = 16


In [39]:
mask_token_index = torch.where(torch.FloatTensor(testdata['input_ids'][0]) == tokenizer.mask_token_id)[0]
mask_token_index

tensor([7])

In [40]:
mask_token_logits = torch.FloatTensor(temp.predictions)[0, mask_token_index, :]
mask_token_logits

tensor([[-0.3223, -0.4032, -0.2217,  ..., -1.3434, -0.8945, -1.2912]])

In [41]:
top_N_tokens = torch.topk(mask_token_logits, N, dim=1).indices[0].tolist()

In [42]:
for i in top_N_tokens:
  print(tokenizer.decode(i))

approved
accepted
recognized
sanctioned
accredited
